In [2]:
import carla
import random
import time
import numpy as np
import cv2

def main():
    # Connect to the CARLA client
    client = carla.Client('localhost', 2000)
    client.set_timeout(10.0)
    world = client.get_world()

    try:
        # Set synchronous mode
        settings = world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        world.apply_settings(settings)

        # Spawn a vehicle
        blueprint_library = world.get_blueprint_library()
        vehicle_bp = random.choice(blueprint_library.filter('vehicle.*'))
        spawn_points = world.get_map().get_spawn_points()
        spawn_point = random.choice(spawn_points)
        vehicle = world.spawn_actor(vehicle_bp, spawn_point)
        vehicle.set_autopilot(True)

        # Define the rear-facing camera blueprint
        camera_bp = blueprint_library.find('sensor.camera.rgb')
        camera_bp.set_attribute('image_size_x', '800')
        camera_bp.set_attribute('image_size_y', '600')
        camera_bp.set_attribute('fov', '90')

        # Set the camera's relative pokkkjjsition and orientation (facing backward)
        camera_transform = carla.Transform(carla.Location(x=-5.5, z=2.5), carla.Rotation(pitch=-15))
        camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

        # Create a display window
        cv2.namedWindow('Rear-Facing Camera', cv2.WINDOW_AUTOSIZE)

        # Define a callback to process and display the camera feed
        def process_image(image):
            array = np.frombuffer(image.raw_data, dtype=np.uint8)
            array = array.reshape((image.height, image.width, 4))
            array = array[:, :, :3]
            array = array[:, :, ::-1]
            cv2.imshow('Rear-Facing Camera', array)
            cv2.waitKey(1)

        # Start the camera and set the callback
        camera.listen(process_image)

        # Run the simulation for a specified duration
        simulation_time = 20  # seconds
        start_time = time.time()
        while time.time() - start_time < simulation_time:
            world.tick()

    finally:
        # Clean up: stop and destroy actors
        camera.stop()
        camera.destroy()
        vehicle.destroy()
        cv2.destroyAllWindows()
        # Reset synchronous mode
        settings.synchronous_mode = False
        settings.fixed_delta_seconds = None
        world.apply_settings(settings)

if __name__ == '__main__':
    main()


RuntimeError: time-out of 10000ms while waiting for the simulator, make sure the simulator is ready and connected to localhost:2000